In [1]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
disco_T = "/home/carlo/Documents/Neuroinformatics/SilvaLab/disco_T"

In [9]:
experiment = "soumnya"
group_1_animal_names = "C"
group_2_animal_names = "S"
marker = "CFos"
normalization = "Density"
comparison_folder = "CM-SM"

filename = f"{marker.lower()}_{normalization.lower()}.csv"
filename_path = os.path.join(".", "data", "experiments", experiment, "BraiAn_PLS_output", comparison_folder, filename)


In [4]:
experiment = "soumnya"
group_1_animal_names = "C"
group_1_plot_name = "control_females"
group_2_animal_names = "S"
group_2_plot_name = "stress_females"
marker = "CFos"
normalization = "RelativeDensity"
comparison_folder = "CF-SF"

filename = f"{marker.lower()}_{normalization.lower()}.csv"
filename_path = os.path.join(disco_T, "Lab Matteoli", "Silva", "Sowmna", "results", "BraiAn_PLS_output", filename)

In [5]:
# ####################################### SET PARAMETERS ####################################

plots_output_path = os.path.join(".", "plots", experiment, comparison_folder)

# ###########################################################################################

if not(os.path.exists(plots_output_path)):
    os.makedirs(plots_output_path, exist_ok=True)

In [6]:
from scipy.stats import kendalltau, pearsonr, spearmanr

In [7]:
def get_stars(group, p):
    stars = pd.DataFrame(index=p.index, columns=p.columns)
    stars[(~p.isna()) & (p <= 0.05)  & (p > 0.01)] = "*"
    stars[(~p.isna()) & (p <= 0.01)  & (p > 0.001)] = "**"
    stars[(~p.isna()) & (p <= 0.001)] = "***"
    stars[(~p.isna()) & (p > 0.05)] = " "
    return stars

In [10]:
group_1 = pd.read_csv(filename_path, sep=";", usecols=lambda c: group_1_animal_names in c or c == "BrainRegion")
assert "BrainRegion" in group_1.columns, f"'{filename}' is expected to have a column called 'BrainRegion'."
group_1 = group_1.set_index("BrainRegion")
group_1.reindex(index=group_1.index[::-1])
group_1_r = group_1.transpose().corr(method=lambda x,y: pearsonr(x,y)[0], min_periods=4)
group_1_p = group_1.transpose().corr(method=lambda x,y: pearsonr(x,y)[1], min_periods=4)
group_1_stars = get_stars(group_1, group_1_p)

group_2 = pd.read_csv(filename_path, sep=";", usecols=lambda c: group_2_animal_names in c or c == "BrainRegion")
assert "BrainRegion" in group_2.columns, f"'{filename}' is expected to have a column called 'BrainRegion'."
group_2 = group_2.set_index("BrainRegion")
group_2.reindex(index=group_2.index[::-1])
group_2_r = group_2.transpose().corr(method=lambda x,y: pearsonr(x,y)[0], min_periods=4)
group_2_p = group_2.transpose().corr(method=lambda x,y: pearsonr(x,y)[1], min_periods=4)
group_2_stars = get_stars(group_2, group_2_p)


In [11]:
def plot_cross_correlation(r, p, stars, filename=None):
    aspect_ratio = 3/2
    cell_height = 18
    cell_width = cell_height*aspect_ratio
    plt_height = cell_height*len(r)
    plt_width = cell_width*len(r)


    fig = go.Figure(data=go.Heatmap(
                        x=r.index,
                        y=r.columns,
                        z=r,
                        text=stars.values,
                        zmin=-1, zmax=1, colorscale="RdBu_r",
                        customdata=np.stack((p,), axis=-1),
                        hovertemplate="%{x} - %{y}<br>r: %{z}<br>p: %{customdata[0]}<extra></extra>",
                        texttemplate="%{text}",
                        #textfont={"size":20}
    ))
    fig.update_layout(width=plt_width, height=plt_height)
    if filename != None:
        fig.write_image(filename)
    return fig

In [12]:
group_1_plot_filename = f"cross_correlation_{marker}_{normalization}_{group_1_plot_name}.svg"
group_1_plot_filepath = os.path.join(plots_output_path, group_1_plot_filename)
# group_1_plot_filepath = None
plot_cross_correlation(group_1_r, group_1_p, group_1_stars, filename=group_1_plot_filepath)

In [13]:
group_2_plot_filename = f"cross_correlation_{marker}_{normalization}_{group_2_plot_name}.svg"
group_2_plot_filepath = os.path.join(plots_output_path, group_2_plot_filename)
# group_2_plot_filepath = None
plot_cross_correlation(group_2_r, group_2_p, group_2_stars, filename=group_2_plot_filepath)